<a href="https://colab.research.google.com/github/kumar045/Assignment-For-Filed/blob/main/ludwig_llama2_7b_finetuning_4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama2-7b Fine-Tuning with 4bit Quantization

We recommend using a GPU runtime for this example. In the Colab menu bar, choose Runtime > Change Runtime Type and choose GPU under Hardware Accelerator.

## Install Ludwig

We'll use the latest version of Ludwig which includes support for quantized fine-tuning.

In [ ]:
!pip uninstall -y tensorflow --quiet
!pip install "git+https://github.com/ludwig-ai/ludwig.git#egg=ludwig[llm]" --quiet

## Set your HuggingFace API Token

Obtain a [HuggingFace API Token](https://huggingface.co/docs/hub/security-tokens) and request access to [Llama2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf) before proceeding.

In [ ]:
import os

os.environ["HUGGING_FACE_HUB_TOKEN"] = "<api_token>"

## Configure Model Training

The Ludwig [configuration](https://ludwig.ai/latest/configuration/) specifies the components of the training job including:

- Model type (LLM) and base pretrained model name from HuggingFace
- Input and output features from the training dataset
- Quantization (4bit) and parameter-efficient fine-tuning (LoRA)
- Training hyperparameters (learning rate, batch size, etc.)
- Preprocessing (e.g., sampling to speed up training)
- Backend for execution (local, but could also be Ray)

In [ ]:
import yaml

config_str = """
model_type: llm
base_model: meta-llama/Llama-2-7b-hf

input_features:
  - name: instruction
    type: text

output_features:
  - name: output
    type: text

quantization:
  bits: 4

adapter:
  type: lora

trainer:
  type: finetune
  learning_rate: 0.0003
  batch_size: 1
  gradient_accumulation_steps: 8
  epochs: 3
  learning_rate_scheduler:
    warmup_fraction: 0.01

preprocessing:
  sample_ratio: 0.01

backend:
  type: local
"""

config = yaml.safe_load(config_str)

## Train!

Start training on your local GPU and monitor progress (including metrics) inline.

In this example, we'll be training on the [Alpaca](https://crfm.stanford.edu/2023/03/13/alpaca.html) dataset to turn Llama2-7b into a rudimentary chatbot. But you can use any dataset to fine-tune for other tasks.

In [ ]:
import logging
from ludwig.api import LudwigModel


model = LudwigModel(config=config, logging_level=logging.INFO)
results = model.train(dataset="ludwig://alpaca")
print(results)

## What's next?

Now you can save / export your model to HuggingFace or explore [Predibase](https://predibase.com/) for serving and managed infrastructure for training larger LLMs and other model types.